<a href="https://colab.research.google.com/github/drkulkarni236/w266_finalproject/blob/main/Deva_BERT_Trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Huggingface example

In [2]:
#https://huggingface.co/transformers/v3.2.0/custom_datasets.html#sequence-classification-with-imdb-reviews
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

--2022-02-13 22:43:46--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  17.0MB/s    in 8.2s    

2022-02-13 22:43:55 (9.82 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [79]:
from pathlib import Path

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir is "neg" else 1)

    return texts, labels

train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')


In [80]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [81]:
#Subset data set to 5000 train and 500 val
train_texts = train_texts[:5000]
train_labels = train_labels[:5000]
val_texts = val_texts[:500]
val_labels = val_labels[:500]

In [82]:
!pip install transformers
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [83]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [10]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

In [84]:
from transformers import TFDistilBertForSequenceClassification

model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=tf.metrics.SparseCategoricalAccuracy(),
# )

optimizer = tf.keras.optimizers.Adam(learning_rate=10e-5)

model.compile(optimizer=optimizer, 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=tf.metrics.SparseCategoricalAccuracy()) # can also use any keras loss fn

model.fit(train_dataset.shuffle(1000).batch(16), validation_data=val_dataset.shuffle(1000).batch(16), epochs=1, batch_size=16)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_projector', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_113', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use 

 20/234 [=>............................] - ETA: 3:31 - loss: 0.7003 - sparse_categorical_accuracy: 0.4844

KeyboardInterrupt: ignored

#Import Hindi Data

In [4]:
from google.colab import files
uploaded = files.upload()

Saving hindi_dataset.tsv to hindi_dataset.tsv


In [5]:
uploaded2 = files.upload()

Saving hasoc2019_hi_test_gold_2919.tsv to hasoc2019_hi_test_gold_2919.tsv


In [6]:
import pandas as pd
import io

df_train = pd.read_csv(io.StringIO(uploaded['hindi_dataset.tsv'].decode('utf-8')), sep = '\t')

In [8]:
df_test = pd.read_csv(io.StringIO(uploaded2['hasoc2019_hi_test_gold_2919.tsv'].decode('utf-8')), sep = '\t')

#BERT as a blackbox notebook

In [49]:
import pandas as pd
url = 'https://raw.githubusercontent.com/drkulkarni236/w266_finalproject/main/Data/hindi_dataset/hindi_dataset.tsv'
df_train = pd.read_csv(url, on_bad_lines='skip', sep='\t')

url = 'https://raw.githubusercontent.com/drkulkarni236/w266_finalproject/main/Data/hindi_dataset/hasoc2019_hi_test_gold_2919.tsv'
df_test = pd.read_csv(url, on_bad_lines='skip', sep='\t')

In [29]:
df_train.head(5)

,text_id,text,task_1,task_2,task_3
0,hasoc_hi_5556,"बांग्लादेश की शानदार वापसी, भारत को 314 रन पर ...",NOT,NONE,NONE
1,hasoc_hi_5648,सब रंडी नाच देखने मे व्यस्त जैसे ही कोई #शांती...,HOF,PRFN,UNT
2,hasoc_hi_164,तुम जैसे हरामियों के लिए बस जूतों की कमी है शु...,HOF,PRFN,TIN
3,hasoc_hi_3530,"बीजेपी MLA आकाश विजयवर्गीय जेल से रिहा, जमानत ...",NOT,NONE,NONE
4,hasoc_hi_5206,चमकी बुखार: विधानसभा परिसर में आरजेडी का प्रदर...,NOT,NONE,NONE


In [30]:
df_test.head(5)

,text_id,text,task_1,task_2,task_3
0,hasoc_hi_5061,"वक्त, इन्सान और इंग्लैंड का मौसम आपको कभी भी ध...",NOT,NONE,NONE
1,hasoc_hi_2090,#कांग्रेस के इस #कमीने की #करतूत को देखिए देश ...,HOF,OFFN,TIN
2,hasoc_hi_2960,पाकिस्तान को फेकना था फेका गया। जो हार कर भी द...,HOF,OFFN,TIN
3,hasoc_hi_864,जो शब्द तूम आज किसी और औरत के लिए यूज कर रहे व...,NOT,NONE,NONE
4,hasoc_hi_54,नेता जी हम समाजवादी सिपाही हमेशा आपके साथ है आ...,NOT,NONE,NONE


In [31]:
print(df_train.shape)
print(df_test.shape)

(4665, 5)
(1318, 5)


In [32]:
!pip install -q transformers
#!pip install transformers

In [96]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

In [34]:
# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True)

# # tokenized_train = df_train.map(tokenize_function, batched=True)
# # tokenized_test = df_test.map(tokenize_function, batched=True)

AttributeError: ignored

In [36]:
df_train.head(2)

,text_id,text,task_1,task_2,task_3
0,hasoc_hi_5556,"बांग्लादेश की शानदार वापसी, भारत को 314 रन पर ...",NOT,NONE,NONE
1,hasoc_hi_5648,सब रंडी नाच देखने मे व्यस्त जैसे ही कोई #शांती...,HOF,PRFN,UNT


In [97]:
train_texts = list(df_train.text)
train_labels = list((df_train.task_1 == 'HOF').astype(int))

test_texts = list(df_test.text)
test_labels = list((df_test.task_1 == 'HOF').astype(int))

In [98]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [99]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [100]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

In [101]:
train_encodings[0]

Encoding(num_tokens=311, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [102]:
from transformers import TFDistilBertForSequenceClassification

model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')


# from transformers import TFBertModel

# model = TFBertModel.from_pretrained('distilbert-base-multilingual-cased')

# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=tf.metrics.SparseCategoricalAccuracy(),
# )

optimizer = tf.keras.optimizers.Adam(learning_rate=10e-5)

model.compile(optimizer=optimizer, 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=tf.metrics.SparseCategoricalAccuracy()) # can also use any keras loss fn

model.fit(train_dataset.shuffle(1000).batch(16), validation_data=val_dataset.shuffle(1000).batch(16), epochs=1, batch_size=16)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_projector', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_170', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use 

234/234 [==============================] - 273s 1s/step - loss: 0.6351 - sparse_categorical_accuracy: 0.6404 - val_loss: 0.5866 - val_sparse_categorical_accuracy: 0.6977
